In [ ]:
import os
from getpass import getpass
os.environ["ASTRA_DB_API_ENDPOINT"] = ""
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = ""
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("Enter your Hugging Face Token: ")

Enter your Hugging Face Token: ··········


In [2]:
from langchain_astradb import AstraDBVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vstore = AstraDBVectorStore(
    collection_name = "demo_project",
    embedding=embeddings,
    token = os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT")
)
print("Print Astra vector DB Configured!!!")

/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Print Astra vector DB Configured!!!


In [3]:
from datasets import load_dataset

philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [4]:
from langchain.schema import Document
## Constructing the set of documents from the dataset
docs=[]
for entry in philo_dataset:
  metadata = {"author": entry["author"]}
  if entry["tags"]:
    for tag in entry["tags"].split(";"):
      metadata[tag] = "y"
    doc = Document(page_content=entry["quote"], metadata = metadata)
    docs.append(doc)

In [5]:
docs[1]

Document(metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}, page_content='The roots of education are bitter, but the fruit is sweet.')

In [6]:
inserted_ids = vstore.add_documents(docs)
print(f"\n Inserted {len(inserted_ids)} documents.")
print(inserted_ids)


 Inserted 292 documents.
['4ff2607fba7748238c79e9310e4bfc74', '2407e0b7057247c6be3a071f1ba2d542', 'a1a75703b4d74b8a998c1e701b921d0a', 'a26dbcb59b3640bba235affd2fa69621', '5daacc83ba6f413a83ec7dfd09501f76', '5f18c8ee34254748baa4f7490f0bc5e2', 'd1e3a10e59ef43638e18051ae0ca4e06', '7fb40ba73b2d4837aee7a96e1a653697', '9d26d7d3dce34d9188c5fe5e8da3d215', 'dd2a32b7693b490b9c3ab41dd936297f', 'a15dfa2283d54283be4ca49ab7126fb3', 'c50b107514dd42edabc85eb3598d9d7d', 'edb208626a87478b8b82ff71f24ebfe9', '7f054050b8a74f7a95fa191cdb0caaa8', '4ff2e1f17db54c43b870d069cdad0274', '471afc9105f542fb9d7605bccb95193d', '8951ff25382a48d69aee288fd451900a', 'bb6c87e737fa4619b77cab90634075f3', 'b9417775905d4351958ac7dc0b68bc9d', 'a12a4b960b7441ae9499bc542dc8ed33', 'dd9e1cd078c64871a7d66c495661fb85', '9540f31adb124a11b2972c7f191ca984', 'b8be19a542cf4e039e4b8d186e477b59', 'db2a6e2e10b64426b1188fc817fd6578', '08930ef325554ac69680cdabc257aca2', '76ab1ae1839a4d038d47e1523b66cf5f', '380e2a0a603b42b499cba3047f00250f', '

In [7]:
print(vstore.astra_db.collection("demo_project").find())

{'data': {'documents': [{'_id': '2afe5f2c6f0a429a8e4fee4350dc70cb', 'content': 'One cannot make a slave of a free person, for a free person is free even in a prison.', 'metadata': {'author': 'plato', 'ethics': 'y', 'knowledge': 'y'}}, {'_id': '4e1bf4017aed45b7ac0825c2c525a81c', 'content': 'You should not honor men more than truth.', 'metadata': {'author': 'plato', 'ethics': 'y'}}, {'_id': 'a2f240c0a436419599aa46c3f6ca5664', 'content': 'World history is a court of judgment.', 'metadata': {'author': 'hegel', 'history': 'y'}}, {'_id': '9b1401a74bd645eeb26123fe7a0e18bf', 'content': 'Render a kiss or blow', 'metadata': {'author': 'sartre', 'love': 'y'}}, {'_id': '2407e0b7057247c6be3a071f1ba2d542', 'content': 'The roots of education are bitter, but the fruit is sweet.', 'metadata': {'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}}, {'_id': 'bb29bc47912c4c2e9906bd0df36d4f6e', 'content': 'Everything the State says is a lie, and everything it has it has stolen.', 'metadata': {'author

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from transformers import pipeline

retriever = vstore.as_retriever(serach_kwargs={"k":3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
 Context: {context}
 Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
pipe = pipeline(
    "text-generation",
    model="tiiuae/falcon-7b-instruct",  # smaller variants if CPU is slow
    device=0  # CPU; set device=0 for GPU runtime in Colab
)
model = HuggingFacePipeline(pipeline=pipe)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what subject are philosophers most concerned with")